# Plafonnement de l'avantage du quotient conjugal dans le calcul de l'IRPP

In [1]:
from datetime import date

from openfisca_france import init_country
from openfisca_france.model.base import *

## Adaptation pour faciliter l'usage de ce notebook

_Ce correctif permet de redéfinir plusieurs fois la même variable sans provoquer d'erreur._

# Variable avec formule

# Variable avec différentes formules en fonction de la date

# Système socio-fiscal

In [2]:
TaxBenefitSystem = init_country()
tax_benefit_system = TaxBenefitSystem()

# Simulation

In [3]:
# age = 60
# ages = [12, 37, 28]

# jean_eric = {
#     'name': u'Jean-Éric',
#     'age': 33,
# }
# aurelie = dict(
#     name = u'Aurélie',
#     age = 33,
# )

# personnes = [aurelie, jean_eric]

In [4]:
scenario = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_de_base = 400000,
        statmarit = 5,
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 0,
        ),
    enfants = [
        ],
    )

simulation_exemple = scenario.new_simulation(debug = True)

In [5]:
simulation_exemple2 = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_de_base = 400000,
        ),
    enfants = [
        ],
    ).new_simulation(debug = True)

In [6]:
print "exemple 1",simulation_exemple.calculate("impo"), \
       ", exemple 2", simulation_exemple2.calculate("impo")

exemple 1 [ 0.] , exemple 2 [-143454.59375]


In [7]:
scenario = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 400000,
        statmarit = 5,
        ),
    )

simulation_exemple = scenario.new_simulation(debug = True)

In [8]:
import json
import urllib
import webbrowser
def get_trace_tool_link(scenario, variables, api_url = u'http://api-test.openfisca.fr',
        trace_tool_url = u'http://www.openfisca.fr/outils/trace'):
    scenario_json = scenario.to_json()
    simulation_json = {
        'scenarios': [scenario_json],
        'variables': variables,
        }
    url = trace_tool_url + '?' + urllib.urlencode({
        'simulation': json.dumps(simulation_json),
        'api_url': api_url,
        })
    return url

In [9]:
print get_trace_tool_link(scenario, ["impo"])

http://www.openfisca.fr/outils/trace?api_url=http%3A%2F%2Fapi-test.openfisca.fr&simulation=%7B%22scenarios%22%3A+%5B%7B%22period%22%3A+%222014%22%2C+%22test_case%22%3A+%7B%22familles%22%3A+%5B%7B%22id%22%3A+0%2C+%22parents%22%3A+%5B%22ind0%22%5D%7D%5D%2C+%22foyers_fiscaux%22%3A+%5B%7B%22id%22%3A+0%2C+%22declarants%22%3A+%5B%22ind0%22%5D%7D%5D%2C+%22individus%22%3A+%5B%7B%22id%22%3A+%22ind0%22%2C+%22salaire_imposable%22%3A+400000.0%2C+%22statmarit%22%3A+5%2C+%22birth%22%3A+%221980-01-01%22%7D%5D%2C+%22menages%22%3A+%5B%7B%22id%22%3A+0%2C+%22personne_de_reference%22%3A+%22ind0%22%7D%5D%7D%7D%5D%2C+%22variables%22%3A+%5B%22impo%22%5D%7D


# Réforme

In [10]:
from openfisca_core import reforms

from numpy import (datetime64, logical_and as and_, logical_not as not_, logical_or as or_, logical_xor as xor_,
    maximum as max_, minimum as min_, round)

In [11]:
Reform = reforms.make_reform(
    key = 'plafonnement_gain_quotient_conjugal',
    name = u"Réforme des cotisations pour un Revenu de base",
    reference = tax_benefit_system,
    )

In [12]:
@Reform.formula
class rev_cat_tspr_individuel(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Nouvelles cotisations contributives"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'year').period('year')
        tspr = simulation.calculate('tspr', period)
        indu_plaf_abat_pen = simulation.calculate('indu_plaf_abat_pen_individuel', period)
        return period, tspr + indu_plaf_abat_pen
    
@Reform.formula
class indu_plaf_abat_pen_individuel(SimpleFormulaColumn):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"indu_plaf_abat_pen"

    def function(self, simulation, period):
        """
        Plafonnement de l'abattement de 10% sur les pensions du foyer
        'foy'
        """
        period = period.start.offset('first-of', 'year').period('year')
        abatpen = simulation.legislation_at(period.start).ir.tspr.abatpen

        pen_net = simulation.calculate('pen_net', period)
        rev_pen = simulation.calculate('rev_pen', period)
        print type(pen_net)

        abat = rev_pen - pen_net
        return period, abat - min_(abat, abatpen.max)


In [13]:
@Reform.formula
class rev_cat_individuel(SimpleFormulaColumn):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Revenus catégoriels"
    url = "http://www.insee.fr/fr/methodes/default.asp?page=definitions/revenus-categoriesl.htm"

    def function(self, simulation, period):
        '''
        Revenus Categoriels
        '''
        period = period.start.offset('first-of', 'year').period('year')
        rev_cat_tspr_individuel = simulation.calculate('rev_cat_tspr_individuel', period)
       # rev_cat_rvcm = simulation.calculate('rev_cat_rvcm', period)
       # rev_cat_rfon = simulation.calculate('rev_cat_rfon', period)
       # rev_cat_rpns = simulation.calculate('rev_cat_rpns', period)
       # rev_cat_pv = simulation.calculate('rev_cat_pv', period)

        return period, rev_cat_tspr_individuel # + rev_cat_rvcm + rev_cat_rfon + rev_cat_rpns + rev_cat_pv TODO :Add everything


In [14]:
@Reform.formula
class rbg_individuel(SimpleFormulaColumn):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Revenu brut global"
    url = "http://www.documentissime.fr/dossiers-droit-pratique/dossier-19-l-impot-sur-le-revenu-les-modalites-generales-d-imposition/la-determination-du-revenu-imposable/le-revenu-brut-global.html"

    def function(self, simulation, period):
        '''Revenu brut global
        '''
        period = period.start.offset('first-of', 'year').period('year')
        rev_cat_individuel = simulation.calculate('rev_cat_individuel', period)
       # deficit_ante = simulation.calculate('deficit_ante', period)
      #  f6gh = simulation.calculate('f6gh', period)
        #nbic_impm_individuel = simulation.calculate('nbic_impm', period)
        #nacc_pvce_individuel = simulation.calculate('nacc_pvce', period)
        cga = simulation.legislation_at(period.start).ir.rpns.cga_taux2

        # (Total 17)
        # sans les revenus au quotient
        #nacc_pvce = self.sum_by_entity(nacc_pvce_holder)
        return period, max_(0, rev_cat_individuel) # + f6gh + nbic_impm_individuel + nacc_pvce) * (1 + cga) - deficit_ante


In [15]:
@Reform.formula
class rng_individuel(SimpleFormulaColumn):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Revenu net global"
    url = "http://impotsurlerevenu.org/definitions/114-revenu-net-global.php"

    def function(self, simulation, period):
        ''' Revenu net global (total 20) '''
        period = period.start.offset('first-of', 'year').period('year')
        rbg_individuel = simulation.calculate('rbg_individuel', period)
        csg_deduc = simulation.calculate('csg_deduc', period)
        #print csg_deduc  #TODO : CHeck that, why is it a Foyer size ?
        charges_deduc = simulation.calculate('charges_deduc', period)

        return period, max_(0, rbg_individuel - csg_deduc - charges_deduc)


In [16]:
@Reform.formula
class rni_individuel(SimpleFormulaColumn):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Revenu net imposable"
    url = "http://impotsurlerevenu.org/definitions/115-revenu-net-imposable.php"

    def function(self, simulation, period):
        ''' Revenu net imposable ou déficit à reporter'''
        period = period.start.offset('first-of', 'year').period('year')
        rng_individuel = simulation.calculate('rng_individuel', period)
        #abat_spe = simulation.calculate('abat_spe', period)

        return period, rng_individuel #- abat_spe

In [17]:
@Reform.formula
class ir_brut_individuel(SimpleFormulaColumn):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Impot sur le revenu brut avant non imposabilité et plafonnement du quotient"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('year')
        nbptr_holder = simulation.compute('nbptr', period)
        nbptr_individualise = self.cast_from_entity_to_roles(nbptr_holder)
        nb_adult = simulation.calculate('nb_adult', period)
        nbptr_individuel = nbptr_individualise * (nb_adult == 1) + (nbptr_individualise/2) * (nb_adult == 2)    
        taux_effectif = simulation.calculate('taux_effectif', period)
        rni_individuel = simulation.calculate('rni_individuel', period)
        bareme = simulation.legislation_at(period.start).ir.bareme

        return period, (taux_effectif == 0) * nbptr_individuel * bareme.calc(rni_individuel / nbptr_individualise) + \
                        taux_effectif * rni_individuel

In [44]:
@Reform.formula
class ir_brut_individuel_foyer(SimpleFormulaColumn):
    column = FloatCol(default = 0)
    entity_class = FoyersFiscaux
    label = u""

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('year')
        ir_brut_individuel_holder = simulation.compute('ir_brut_individuel', period)
        ir_brut_individuel_foyer = self.sum_by_entity(ir_brut_individuel_holder)
        return period, ir_brut_individuel_foyer

Caution: You are adding a formula to an instantiated Reform. Reform must be reinstatiated.


In [36]:
@Reform.formula
class ir_qf_conjugual_plafonne_revenu_de_base(SimpleFormulaColumn):
    column = FloatCol(default = 0)
    entity_class = FoyersFiscaux
    label = u""

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('year')
        seuil = 6032
        ir_brut = simulation.calculate('ir_brut', period)
        ir_brut_individuel_holder = simulation.compute('ir_brut_individuel', period)
        ir_indivudel_somme = self.sum_by_entity(ir_brut_individuel_holder)
        ir_qf_conj_plaf = min_(ir_indivudel_somme- seuil, ir_brut )
        return period, ir_qf_conj_plaf
        

Caution: You are adding a formula to an instantiated Reform. Reform must be reinstatiated.


In [37]:
[2,3,4]

[2, 3, 4]

In [45]:
reform = Reform()

In [46]:
reform_simulation =  reform.new_scenario().init_single_entity(
                                                                period = 2014,
                                                                parent1 = dict(
                                                                    birth = date(1980, 1, 1),
                                                                    salaire_imposable = 400000,
                                                                    ),
                                                                parent2 = dict(
                                                                    birth = date(1980, 1, 1),
                                                                    salaire_imposable = 100000,
                                                                    ),
                                                                enfants = [
                                                                    ],
                                                                ).new_simulation(debug = True)

In [52]:
print reform_simulation.calculate('ir_qf_conjugual_plafonne_revenu_de_base')
print reform_simulation.calculate('ir_brut_individuel_foyer')
print reform_simulation.calculate('ir_qf_conjugual_plafonne_revenu_de_base')
reform_simulation.calculate('ir_qf_conjugual_plafonne_revenu_de_base') - reform_simulation.calculate('ir_brut_individuel_foyer')

[ 172255.546875]
[ 178287.546875]
[ 172255.546875]


array([-6032.], dtype=float32)

In [ ]:
reform_simulation.calculate('rev_cat_individuel')

In [48]:
reform_simulation.calculate('ir_brut_individuel_foyer')

array([ 178287.546875], dtype=float32)

In [21]:
reform_simulation.calculate('cotisations_contributives')

KeyError: 'cotisations_contributives'

In [ ]:
reform_simulation.calculate('nouv_salbrut')

In [ ]:
reform_simulation.calculate('salbrut')

In [ ]:
reform_simulation.calculate('salnet')

[u'nbN',
 u'nbR',
 u'caseE',
 u'caseF',
 u'caseG',
 u'caseH',
 u'caseK',
 u'caseL',
 u'caseN',
 u'caseP',
 u'caseS',
 u'caseT',
 u'caseW',
 u'b1ab',
 u'b1ac',
 u'b1bc',
 u'b1be',
 u'b1bh',
 u'b1bk',
 u'b1cl',
 u'b1cb',
 u'b1cd',
 u'b1ce',
 u'b1cf',
 u'b1cg',
 u'b1co',
 u'b2gh',
 u'b2mt',
 u'b2ne',
 u'b2mv',
 u'b2nf',
 u'b2mx',
 u'b2na',
 u'b2nc',
 u'b4rs',
 u'rev_or',
 u'rev_exo',
 u'tax_fonc',
 u'restit_imp',
 u'isf_imm_bati',
 u'isf_imm_non_bati',
 u'isf_actions_sal',
 u'isf_droits_sociaux',
 u'ass_isf',
 u'isf_iai',
 u'isf_avant_reduction',
 u'isf_reduc_pac',
 u'isf_inv_pme',
 u'isf_org_int_gen',
 u'isf_avant_plaf',
 u'tot_impot',
 u'revetproduits',
 u'decote_isf',
 u'isf_apres_plaf',
 u'isf_tot',
 u'rvcm_plus_abat',
 u'maj_cga',
 u'bouclier_rev',
 u'bouclier_imp_gen',
 u'restitutions',
 u'bouclier_sumimp',
 u'bouclier_fiscal',
 u'f6de',
 u'f6gi',
 u'f6gj',
 u'f6el',
 u'f6em',
 u'f6gp',
 u'f6gu',
 u'f6eu',
 u'f6ev',
 u'f6dd',
 u'f6aa',
 u'f6cc',
 u'f6eh',
 u'f6da',
 u'f6cb',
 u'f6hj

In [ ]:
tax_benefit_system.column_by_name.keys()